In [288]:
import tensorflow as tf
import numpy as np
import json
import pickle
import pandas as pd
from processing import batch_generator, dataTest,build_dataset
from keras.preprocessing.sequence import pad_sequences
import random
model = tf.keras.models.load_model("file/CNN_LSTM4_10_model")

In [289]:
data_dir=["file/pre_datas_test1.csv","file/pre_datas_test2.csv","file/pre_datas_test3.csv",
                "file/pre_datas_test4.csv","file/pre_datas_test5.csv","file/pre_datas_test6.csv",
                "file/pre_datas_test7.csv","file/pre_datas_test8.csv","file/pre_datas_test9.csv",
                "file/pre_datas_test10.csv"]
model_dir="file/CNN_LSTM4_2_model"


In [290]:
batch_size=100
train_generators, test_generators, train_size, test_size, input_num, dims_num= build_dataset(batch_size)

In [291]:
vec_dir="file/word2vec.pickle"

with open(vec_dir, "rb") as f:
        word2vec = pickle.load(f)
dictionary=word2vec["dictionary"]
embeddings = word2vec["embeddings"]
reverse_dictionary = word2vec["reverse_dictionary"]
train_size=word2vec["train_size"]
test_size=word2vec["test_size"]
dims_num = word2vec["dims_num"]
input_num =word2vec["input_num"]


In [292]:
from utils import GeneSeg
x="prodid%3D541501%27%22--%3E%3C/style%3E%3C/script%3E%3Cscript%3Ealert%28%22XSS%22%29%3C/script%3E%26site%3D642%26from%3D3201552%26noreg%3DY%26clear%3DY%26referer1%3Dkl_main%26referer2%3Dkl_store"
x=GeneSeg(x)
print(x)

['prodid=', '0', '>', '</style>', '</script>', '<script>', 'alert(', '"xss"', ')', '</script>', 'site=', '0', 'from=', '0', 'noreg=', 'y', 'clear=', 'y', 'referer0=', 'kl_main', 'referer0=', 'kl_store']


In [293]:
def to_index(data):
        d_index=[]
        for word in data:
            if word in dictionary.keys():
                d_index.append(dictionary[word])
            else:
                d_index.append(dictionary["UNK"])
        return d_index

In [294]:
datas_index=[to_index(data) for data in x]
datas_index=pad_sequences(datas_index,value=-1)
#print(datas_index)
rand=random.sample(range(len(datas_index)),len(datas_index))
datas=[datas_index[index] for index in rand]
"""print(datas)
print(len(datas))
print(len(datas[0]))
"""
appendingData=[]
print(len(datas[0]))
counter=755-len(datas[0])
for i in range(counter):
    appendingData.append(-1)
print(len(appendingData))
#print(appendingData)
#print(len(appendingData))
#appendingData=pd.array(appendingData,dtype="int32"
#print(appendingData)
#print(appendingData)
#print(datas)
appendingDataCopy=appendingData
print(len(appendingDataCopy))
for j in range(len(datas)):
    for i in range(len(datas[j])):
        appendingDataCopy.append(datas[j][i])
    
    #print("hi : ",len(appendingDataCopy))
    #print(datas[j])
    datas[j]=appendingDataCopy
   # print(datas[j])
    #print(datas[j])
    appendingDataCopy=[]
    for w in range(counter):
        appendingDataCopy.append(-1)



    
#print(len(datas[0]))
"""for i in range(len(datas)):
    df1 = pd.DataFrame(appendingData)
    df2=pd.DataFrame(datas[i])
    df1.append(df2)
    datas[i]=pd.array(df1,dtype="int32")
"""
#print(datas[0])
w=np.asarray(datas,dtype='int32')
#print(w[0:2])

#print(datas)

9
746
746


In [295]:
with open("PredictionDataTest.csv","w") as f:
            for i in range(len(w)):
                data_line=str(w[i].tolist())+"\n"
                f.write(data_line)

In [296]:
print(train_size)
print(test_size)

94923
10547


In [297]:
def data_generator(data_dir):
    #value = tf.data.TextLineDataset([data_dir])
    #reader = tf.TextLineReader()
    #queue = tf.train.string_input_producer([data_dir])
    #_, value = reader.read(queue)
    #coord = tf.train.Coordinator()
    #sess = tf.Session()
    #threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    #while True:
    #    v = value
    #    [data, label] = v.split(b"|")
    #    data = np.array(json.loads(data.decode("utf-8")))
    #    label = np.array(json.loads(label.decode("utf-8")))
    #    yield (data, label)
    #coord.request_stop()
    #coord.join(threads)
    #sess.close()
    df = tf.data.TextLineDataset([data_dir])
    #print(df)
    for line in df:
        try:
            [data, label] = tf.strings.split(line, b"|").numpy()
        except:
            [data] = tf.strings.split(line, b"|").numpy()
        data = np.array(json.loads(data.decode("utf-8")))
        label = np.array([])
        yield (data, label)

In [298]:
def batch_generator(datas_dir,datas_size,batch_size,embeddings,reverse_dictionary,train=True):
    batch_data = []
    batch_label = []
    generator=data_generator(datas_dir)
    n=0
    while True:
        for i in range(batch_size):
            data,label=next(generator)
            data_embed = []
            for d in data:
                if d != -1:
                    data_embed.append(embeddings[reverse_dictionary[d]])
                else:
                    data_embed.append([0.0] * len(embeddings["UNK"]))
            batch_data.append(data_embed)
            batch_label.append(label)
            n+=1
            if not train and n==datas_size:
                break
        if not train and n == datas_size:
            yield (np.array(batch_data), np.array(batch_label))
            break
        else:
            yield (np.array(batch_data),np.array(batch_label))
            batch_data = []
            batch_label = []

In [299]:
test12=batch_generator("PredictionDataTest.csv",len(w),1,embeddings,reverse_dictionary,train=False)

In [300]:
# res = []
# for batch in test12:
#    result= model.predict_on_batch(batch)
#    res.append(result)
# arr=result
arr = model.predict(test12)


In [301]:
print(len(arr))
# print(arr)
yes = 0
no = 0
for x in range(len(arr)):
    yes+= arr[x][0]
    no += arr[x][1]
print( "attack : ", yes/len(arr), " \nnot attack : ",no/len(arr))

22
attack :  0.46827878817831253  
not attack :  0.5317212128165093


In [302]:
test_generators=[]
for i in range(10):
    test_generator = batch_generator(data_dir[i],test_size,batch_size,embeddings,reverse_dictionary,train=False)
    test_generators.append(test_generator)

In [303]:
print(batch_size)
test_generator = batch_generator(data_dir[1],test_size,batch_size,embeddings,reverse_dictionary,train=False)

100


In [304]:
# xp=model.predict(test_generator)

In [305]:


# dataTest(model_dir, test_generators[0], test_size, input_num, dims_num, 100)

In [306]:
print(xp)

NameError: name 'xp' is not defined

[array([1.251409e-05, 9.999875e-01], dtype=float32),
 array([3.4060624e-05, 9.9996591e-01], dtype=float32),
 array([9.991959e-01, 8.041584e-04], dtype=float32),
 array([2.0000742e-05, 9.9997997e-01], dtype=float32),
 array([1.987775e-05, 9.999801e-01], dtype=float32),
 array([0.9756477 , 0.02435233], dtype=float32),
 array([9.9980456e-01, 1.9547871e-04], dtype=float32),
 array([1.4952906e-05, 9.9998510e-01], dtype=float32),
 array([9.9970692e-01, 2.9302255e-04], dtype=float32),
 array([9.992823e-01, 7.176720e-04], dtype=float32),
 array([0.9989039 , 0.00109607], dtype=float32),
 array([1.975582e-04, 9.998024e-01], dtype=float32),
 array([1.5444923e-05, 9.9998450e-01], dtype=float32),
 array([0.8379096 , 0.16209036], dtype=float32),
 array([3.3173361e-05, 9.9996686e-01], dtype=float32),
 array([2.994519e-05, 9.999701e-01], dtype=float32),
 array([0.00620435, 0.99379563], dtype=float32),
 array([0.9966846, 0.0033154], dtype=float32),
 array([7.1485985e-05, 9.9992847e-01], dtype=float32),

In [ ]:
count=0
count2=0
for i in w:
    if i[0]>i[1]:
       # print("attack")
        count+=1
    else:
        #print("not attack")
        count2+=1
print("Total attacks:",count," not attcks:",count2)

Total attacks: 3198  not attcks: 7349
